In [41]:
import plotly.graph_objects as go 
import pandas as pd
import ccxt


In [42]:
def fetch_binance_data(symbol='BTC/USDT', timeframe='1d'):
    exchange = ccxt.binance()
    since = exchange.parse8601('2024-01-01T00:00:00Z')
    all_ohlcv = []
    while True:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=since, limit=1000)
        if not ohlcv:
            break
        all_ohlcv.extend(ohlcv)
        since = ohlcv[-1][0] + 1
    
    df = pd.DataFrame(all_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

In [43]:
df = fetch_binance_data()

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  445 non-null    int64         
 1   open       445 non-null    float64       
 2   high       445 non-null    float64       
 3   low        445 non-null    float64       
 4   close      445 non-null    float64       
 5   volume     445 non-null    float64       
 6   datetime   445 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 24.5 KB


In [45]:
import plotly.graph_objects as go 
from datetime import datetime

fig = go.Figure(data = [go.Ohlc(
    x = df['datetime'], 
    open = df['open'], 
    high = df['high'], 
    low = df['low'],
    close = df['close'])])

fig.show()



In [46]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go 

fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True, row_heights = [0.8, 0.2])

fig.add_trace(go.Candlestick( x = df['datetime'], open = df['open'], close = df['close'], low = df['low'], high = df['high']), row = 1, col = 1)

fig.add_trace(go.Bar(x=df['datetime'], y = df['volume']), row = 2, col = 1)

fig.update_layout(xaxis_title = "Date")

fig.show()

In [47]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# 假設有 OHLC 數據
dates = ["2024-03-01", "2024-03-02", "2024-03-03"]
open_prices = [100, 101, 103]
high_prices = [105, 106, 107]
low_prices = [98, 99, 101]
close_prices = [102, 104, 106]
volumes = [5000, 7000, 6500]

# 創建兩個子圖，K 線圖和成交量圖
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[0.7, 0.3])

# K 線圖
fig.add_trace(go.Candlestick(
    x=dates, open=open_prices, high=high_prices, low=low_prices, close=close_prices,
    name="Candlestick"
), row=1, col=1)

# 成交量圖
fig.add_trace(go.Bar(
    x=dates, y=volumes, name="Volume", marker_color="gray"
), row=2, col=1)

# 設定圖表標題
fig.update_layout(title="Stock Chart with Volume", xaxis_title="Date")
fig.show()



In [48]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 假設有歷史股價資料
dates = pd.date_range("2024-01-01", periods=200)  # 200 天的日期
close_prices = np.random.randint(100, 200, size=200)  # 隨機股價作為範例

# 計算 50 天和 90 天的 SMA 和 EMA
df["SMA_50"] = df["close"].rolling(window=50).mean()  # 50 天簡單移動平均
df["SMA_90"] = df["close"].rolling(window=90).mean()  # 90 天簡單移動平均
df["EMA_50"] = df["close"].ewm(span=50, adjust=False).mean()  # 50 天指數移動平均
df["EMA_90"] = df["close"].ewm(span=90, adjust=False).mean()  # 90 天指數移動平均

# 顯示資料
print(df.head())


       timestamp      open      high       low     close       volume  \
0  1704067200000  42283.58  44184.10  42180.77  44179.55  27174.29903   
1  1704153600000  44179.55  45879.63  44148.34  44946.91  65146.40661   
2  1704240000000  44946.91  45500.00  40750.00  42845.23  81194.55173   
3  1704326400000  42845.23  44729.58  42613.77  44151.10  48038.06334   
4  1704412800000  44151.10  44357.46  42450.00  44145.11  48075.25327   

    datetime  SMA_50  SMA_90        EMA_50        EMA_90  
0 2024-01-01     NaN     NaN  44179.550000  44179.550000  
1 2024-01-02     NaN     NaN  44209.642549  44196.415055  
2 2024-01-03     NaN     NaN  44156.136175  44166.718680  
3 2024-01-04     NaN     NaN  44155.938678  44166.375412  
4 2024-01-05     NaN     NaN  44155.514023  44165.908041  


In [51]:

# 創建 K 線圖
fig = go.Figure(data=[go.Candlestick(
    x=df['datetime'],
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close'],
    name='K線'
)])

# 添加移動平均線
fig.add_trace(go.Scatter(
    x=df['datetime'],
    y=df['SMA_50'],
    mode='lines',
    name='50 日均線',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=df['datetime'],
    y=df['SMA_90'],
    mode='lines',
    name='90 日均線',
    line=dict(color='red', width=2, dash='dash')  # 虛線
))

# 顯示圖表
fig.update_layout(
    title='K 線圖與移動平均線',
    xaxis_title='日期',
    yaxis_title='股價'
)

fig.show()